In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 4.1MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.2MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 
     |████████████████████████████████| 3.3MB 27.7MB/s 


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
import pandas as pd
text = pd.read_csv("stockholm/bert_code/trofix_bert_subs/trofix_tp.csv")

In [ ]:
text.shape

(63, 6)

In [ ]:
text.to_csv(r'trofix_bert_tp.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('stockholm/mlm/preds/trofix_bert_tp.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base').eval()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

In [ ]:
text[:35]

['"When she hears that prisoners sometimes carve their names into logs that float downriver , she grabs her child and runs with her to the river ." 1 she grabs child 1',
 '"The latest truce in the week-old battle , which has killed 188 and injured 534 , was arranged by Syrian President Assad and Iranian President Khamenei ." 1 which killed 188 1',
 '"All agree that the state must diversify its industrial base to escape the recently detrimental reliance on oil , said Ron Jones , chairman of the New Orleans Business Council and president of Louisiana Coca Cola Bottling Ltd ." 1 state escape base 1',
 '"`` It floods the area instead of having hot pinpoints of glaring light . \'\'" 1 it floods area 1',
 '"Next month , A & P will roll out these jams and jellies , now sold only in New York , to its 986 stores in the U.S ." 1 a roll jams 1',
 '"-- The justices refused to hear an antitrust appeal by actor Paul Newman and director George Roy Hill in a suit charging that MCA Inc. and its Univers

In [ ]:
text_sentence = 'When she hears that prisoners sometimes carve their names into logs that float downriver , she grabs her <mask> and runs with her to the river . '
get_all_predictions(text_sentence)

When she hears that prisoners sometimes carve their names into logs that float downriver , she grabs her <mask> and runs with her to the river . 


{'bart': 'friend\nsister\nmother\ndaughter\nlittle',
 'bert': 'sister\ndaughter\nmother\npurse\nbackpack',
 'electra': 'hand\narm\nbag\nhead\ngun',
 'roberta': 'daughter\nson\ndog\nsister\nfriend',
 'xlm': 'hand\nphone\ngun\nbag\nbags',
 'xlnet': 'head\nup\ncoat\ngun\nhand'}

In [ ]:
text_sentence = "The latest truce in the week-old battle , which has killed <mask> and injured 534 , was arranged by Syrian President Assad and Iranian President Khamenei ."
get_all_predictions(text_sentence)

The latest truce in the week-old battle , which has killed <mask> and injured 534 , was arranged by Syrian President Assad and Iranian President Khamenei .


{'bart': 'at\nmore\nover\nnearly\nsome',
 'bert': '700\n400\n500\n300\n600',
 'electra': 'thousands\nhundreds\n50\n40\n60',
 'roberta': 'hundreds\ndozens\nthousands\n130\n235',
 'xlm': '11\n12\n14\n13\n16',
 'xlnet': 'and\nkilled\n43\n5\ndied'}

In [ ]:
text_sentence = "All agree that the state must diversify its industrial <mask> to escape the recently detrimental reliance on oil , said Ron Jones , chairman of the New Orleans Business Council and president of Louisiana Coca Cola Bottling Ltd . "
get_all_predictions(text_sentence)

All agree that the state must diversify its industrial <mask> to escape the recently detrimental reliance on oil , said Ron Jones , chairman of the New Orleans Business Council and president of Louisiana Coca Cola Bottling Ltd . 


{'bart': 'base\nand\nsector\neconomy\nproduction',
 'bert': 'sector\nbase\nresources\nactivities\ncapabilities',
 'electra': 'operations\nization\nsector\ninfrastructure\noutput',
 'roberta': 'base\nsector\nsectors\nmix\neconomy',
 'xlm': 'capacity\nactivities\nproduction\nresources\nsector',
 'xlnet': 'industry\nproduction\nsector\nindustries\nbusiness'}

In [ ]:
text_sentence = "A surprisingly strong October employment report , implying that the economy is n\'t <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said "
get_all_predictions(text_sentence)

A surprisingly strong October employment report , implying that the economy is n't <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said 


{'bart': 'cooling\nslowing\nreally\ntaking\nbeing',
 'bert': 'taking\ncooling\ngoing\ngetting\ntake',
 'electra': 'paying\ncoming\ngoing\nbuying\nslowing',
 'roberta': 'cooling\ntaking\neasing\ncoming\nfalling',
 'xlm': 'going\npaying\ntaking\ncoming\ngetting',
 'xlnet': 'quite\nreally\ngetting\ncoming\npulling'}

In [ ]:
text_sentence = "`` It floods the <mask> instead of having hot pinpoints of glaring light . \'\' "
get_all_predictions(text_sentence)

`` It floods the <mask> instead of having hot pinpoints of glaring light . '' 


{'bart': 'room\nwhole\nentire\narea\nscreen',
 'bert': 'room\nsky\nair\ncity\ninterior',
 'electra': 'world\ncity\nsky\nwater\nair',
 'roberta': 'room\nsky\nscene\nscreen\nworld',
 'xlm': 'sky\nroom\ncity\nhouse\nwater',
 'xlnet': 'the\ninstead\nin\nother\nrather'}

In [ ]:
text_sentence = "Next month , A & P will roll out these <mask> and jellies , now sold only in New York , to its 986 stores in the U.S . "
get_all_predictions(text_sentence)

Next month , A & P will roll out these <mask> and jellies , now sold only in New York , to its 986 stores in the U.S . 


{'bart': 'new\nand\nproducts\nflavors\nfresh',
 'bert': 'fruits\nsweets\ncakes\nfruit\nwines',
 'electra': 's\napples\njeans\nrolls\nnuts',
 'roberta': 'jams\ncookies\ncakes\njuices\nshakes',
 'xlm': 'bags\nshoes\nplates\nflowers\nclothes',
 'xlnet': 'cookies\nbars\nthese\nfruits\nhearts'}

In [ ]:
text_sentence = "-- The justices refused to hear an antitrust appeal by actor Paul Newman and director George Roy Hill in a suit charging that MCA Inc. and its Universal City Studios Inc. unit conspired to fix the <mask> of film revenue paid to artists ."
get_all_predictions(text_sentence)

-- The justices refused to hear an antitrust appeal by actor Paul Newman and director George Roy Hill in a suit charging that MCA Inc. and its Universal City Studios Inc. unit conspired to fix the <mask> of film revenue paid to artists .


{'bart': 'rates\nprice\nrate\nprices\nmarket',
 'bert': 'amount\npercentage\nportion\nshare\nlevel',
 'electra': 'amount\nbulk\nlevel\nbalance\nprice',
 'roberta': 'distribution\namount\nrate\npercentage\nbalance',
 'xlm': 'amount\ncost\nportion\npercentage\nloss',
 'xlnet': 'source\namount\ndistribution\nflow\nshare'}

In [ ]:
text_sentence = "Indiana Rep. Andrew Jacobs , a Democrat who \'s neutral in the presidential-nomination contest , says Mr. Gephardt reminds him of a time in his life when he had to eat a <mask> of dinners at a drive-in restaurant . "
get_all_predictions(text_sentence)

Indiana Rep. Andrew Jacobs , a Democrat who 's neutral in the presidential-nomination contest , says Mr. Gephardt reminds him of a time in his life when he had to eat a <mask> of dinners at a drive-in restaurant . 


{'bart': 'lot\ncouple\nmeal\nwhole\nvariety',
 'bert': 'couple\nlot\nbunch\nseries\nton',
 'electra': 'couple\nlot\nbunch\nhandful\nvariety',
 'roberta': 'lot\ncouple\nseries\nvariety\nnumber',
 'xlm': 'couple\nlot\nvariety\nbunch\nnumber',
 'xlnet': 'lot\nseries\ncouple\nbunch\ngroup'}

In [ ]:
text_sentence = "The result: Some car makers are cooling their hot <mask> ."
get_all_predictions(text_sentence)

The result: Some car makers are cooling their hot <mask> .


{'bart': 'engines\nwheels\nair\ncars\nengine',
 'bert': 'cars\nengines\noil\nwheels\nwater',
 'electra': 'air\nsprings\nwater\ncars\ndogs',
 'roberta': 'spots\nrods\nheels\nwheels\nmarkets',
 'xlm': 'oil\nwater\nair\ncar\nchocolate',
 'xlnet': 'hot\ncold\nheat\ncooling\ncool'}

In [ ]:
text_sentence = "While Mr. Gorbachev preached togetherness yesterday , Romania was kicking Hungary \'s <mask> out of the country . "
get_all_predictions(text_sentence)

While Mr. Gorbachev preached togetherness yesterday , Romania was kicking Hungary 's <mask> out of the country . 


{'bart': 'ambassador\nmost\nforeign\nprime\nformer',
 'bert': 'jews\ntroops\nambassador\ncitizens\npeople',
 'electra': 'head\nway\nback\npeople\nass',
 'roberta': 'ambassador\nCommunists\ndiplomats\npeople\nleaders',
 'xlm': 'back\npeople\ngovernment\nway\npresident',
 'xlnet': 's\nUnion\nway\n’\nown'}

In [ ]:
text_sentence = "Mr. Martineau also says the tractors reflect another Russian approach to manufacturing: `` You could melt down one <mask> and make three American tractors with the amount of metal the Russians have put in it , \'\' he says . "
get_all_predictions(text_sentence)

Mr. Martineau also says the tractors reflect another Russian approach to manufacturing: `` You could melt down one <mask> and make three American tractors with the amount of metal the Russians have put in it , '' he says . 


{'bart': 'of\nAmerican\nRussian\npiece\ntractor',
 'bert': 'tractor\nmachine\nengine\ntire\ntruck',
 'electra': 'car\nmachine\ntruck\ntractor\none',
 'roberta': 'tractor\ncar\nplant\nfactory\nthing',
 'xlm': 'German\nmachine\nAmerican\nsteel\ntruck',
 'xlnet': 'tractor\nengine\ntruck\nfactory\nmachine'}

In [ ]:
text_sentence = "But there were stumbling <mask> from the start . "
get_all_predictions(text_sentence)

But there were stumbling <mask> from the start . 


{'bart': 'blocks\nsteps\nstones\nblock\nmoments',
 'bert': 'blocks\nsteps\ns\nstones\nsigns',
 'electra': 's\nblocks\nproblems\naround\npatterns',
 'roberta': 'blocks\npoints\nstones\nbumps\nsteps',
 'xlm': 's\nquestions\nmoments\nplaces\nproblems',
 'xlnet': 'from\nabout\nright\ndown\naround'}

In [ ]:
text_sentence = "`` A strong opening in platinum coupled with a weak dollar just grabbed <mask> by the horns and took it on up , \'\' he said . "
get_all_predictions(text_sentence)

`` A strong opening in platinum coupled with a weak dollar just grabbed <mask> by the horns and took it on up , '' he said . 


{'bart': 'the\nthis\nit\na\nthat',
 'bert': 'it\nme\ngold\nyou\nthis',
 'electra': 'me\nyou\nit\nus\nhim',
 'roberta': 'gold\nit\nplatinum\nsilver\nmetal',
 'xlm': 'me\nit\nus\nhim\nthem',
 'xlnet': 'grabbed\nby\ngrab\nit\nup'}

In [ ]:
text_sentence = "`` I \'m quite sure there was someone within the IBM structure that could have just reached in and fixed <mask> . \'\' "
get_all_predictions(text_sentence)

`` I 'm quite sure there was someone within the IBM structure that could have just reached in and fixed <mask> . '' 


{'bart': 'it\nthe\nthis\nthat\nwhat',
 'bert': 'it\nthem\nthat\nthis\nus',
 'electra': 'it\nthat\nthem\nme\neverything',
 'roberta': 'it\nthat\nthis\neverything\nthings',
 'xlm': 'it\nthis\nthat\neverything\nup',
 'xlnet': 'the\na\nthis\nthat\nan'}

In [ ]:
text_sentence = "Nevertheless , they continue to pump <mask> into the market . "
get_all_predictions(text_sentence)

Nevertheless , they continue to pump <mask> into the market . 


{'bart': 'billions\nmore\na\nmoney\nthe',
 'bert': 'it\nthem\nproducts\nbeer\nfish',
 'electra': 'money\nback\ncash\nheavily\nwater',
 'roberta': 'money\ncash\nliquidity\nproducts\nfunds',
 'xlm': 'them\nmoney\nit\nthemselves\nproducts',
 'xlnet': 'further\nback\nup\nin\ninto'}

In [ ]:
text_sentence = "During the past 18 months , the Houston-based unit of Texas Air Corp. absorbed several <mask> , in the process inheriting a hodge-podge of different aircraft . "
get_all_predictions(text_sentence)

During the past 18 months , the Houston-based unit of Texas Air Corp. absorbed several <mask> , in the process inheriting a hodge-podge of different aircraft . 


{'bart': 'of\nsmaller\nother\nsmall\nairlines',
 'bert': 'aircraft\nsquadrons\nunits\ncompanies\nairlines',
 'electra': 'aircraft\nplanes\njets\ncompanies\nships',
 'roberta': 'airlines\nplanes\nsubsidiaries\naircraft\ncarriers',
 'xlm': 'technologies\nprojects\nvehicles\nprograms\nmaterials',
 'xlnet': 'persons\nmany\nhundred\npeople\nmore'}

In [ ]:
text_sentence = "`` I do n\'t think anyone liked the Nissan campaign , but they went out of their way not to knock <mask> , \'\' says Eli Bloom , the New York GM dealer who sits on the selection committee . -LRB- Mr . "
get_all_predictions(text_sentence)

`` I do n't think anyone liked the Nissan campaign , but they went out of their way not to knock <mask> , '' says Eli Bloom , the New York GM dealer who sits on the selection committee . -LRB- Mr . 


{'bart': 'the\nGM\non\nus\nout',
 'bert': 'it\nme\nyou\naround\nthem',
 'electra': 'it\nout\ndown\nthem\nback',
 'roberta': 'it\nus\nthem\nanybody\nme',
 'xlm': 'out\ndown\non\nit\nme',
 'xlnet': 'it\nthem\nus\nanybody\nme'}

In [ ]:
text_sentence = "`` That remark -LRB- of the official -RRB- definitely cooled the <mask> of the bulls , \'\' said Paul Clohesy , chief dealer at Australia and New Zealand Banking Group Ltd. in New York . "
get_all_predictions(text_sentence)

`` That remark -LRB- of the official -RRB- definitely cooled the <mask> of the bulls , '' said Paul Clohesy , chief dealer at Australia and New Zealand Banking Group Ltd. in New York . 


{'bart': 'fire\nenthusiasm\nmarket\nmood\nappetite',
 'bert': 'mood\nspirits\nspirit\ntemper\nhearts',
 'electra': 'spirits\nminds\nblood\nbutt\nmood',
 'roberta': 'appetite\nenthusiasm\ntemper\nmood\npulse',
 'xlm': 'back\nmood\nrest\nmind\nhead',
 'xlnet': 'temper\nhead\ntail\nspirits\nfeet'}

In [ ]:
text_sentence = "The subject is Oedipus \'s death , and perhaps because death is the one thing that still fills <mask> with awe , the excellence of this scene transmits a shiver of the tragedy \'s original force . "
get_all_predictions(text_sentence)

The subject is Oedipus 's death , and perhaps because death is the one thing that still fills <mask> with awe , the excellence of this scene transmits a shiver of the tragedy 's original force . 


{'bart': 'the\nus\na\nShakespeare\nthis',
 'bert': 'me\nus\nhim\none\nher',
 'electra': 'me\nus\nyou\nit\none',
 'roberta': 'us\nme\nhim\none\nyou',
 'xlm': 'me\nus\nhim\npeople\nyou',
 'xlnet': 'us\nme\none\nyou\nhumanity'}

In [ ]:
text_sentence = "The issue is particularly hot in Florida , where a coalition of civic leaders , civil libertarians and Hispanic groups went to federal court recently in an unsuccessful attempt to knock the proposed <mask> off the ballot . "
get_all_predictions(text_sentence)

The issue is particularly hot in Florida , where a coalition of civic leaders , civil libertarians and Hispanic groups went to federal court recently in an unsuccessful attempt to knock the proposed <mask> off the ballot . 


{'bart': 'constitutional\nban\namendment\nlaw\nlanguage',
 'bert': 'measure\namendment\nbill\nlegislation\nlaw',
 'electra': 'bill\nconstitution\nissue\nproposal\nlegislation',
 'roberta': 'measure\namendment\nreferendum\ntax\nmeasures',
 'xlm': 'bill\nlaw\nresolution\nlaws\nmeasure',
 'xlnet': 'Florida\nlaw\ninitiative\nreferendum\nstate'}

In [ ]:
text_sentence = "Three former scholarship winners now fly border <mask> for the U.S. Customs Service . "
get_all_predictions(text_sentence)

Three former scholarship winners now fly border <mask> for the U.S. Customs Service . 


{'bart': 'patrol\npatrols\ncontrol\nsecurity\nguards',
 'bert': 'patrol\npatrols\nchecks\nmissions\ncrossings',
 'electra': 'flights\nlines\nsigns\nroutes\nvisas',
 'roberta': 'patrol\nsecurity\npatrols\nchecks\nguards',
 'xlm': 's\npatrol\nsecurity\nless\ning',
 'xlnet': 'for\nto\nhere\nin\nwith'}

In [ ]:
text_sentence = "There was market speculation yesterday that senior officials of Olympia & York may have met secretly with Michael D. Dingman , chairman and chief executive officer of Henley , in an effort to smooth over a <mask> between the two one-time suitors for Santa Fe , a Chicago-based railroad and energy concern . "
get_all_predictions(text_sentence)

There was market speculation yesterday that senior officials of Olympia & York may have met secretly with Michael D. Dingman , chairman and chief executive officer of Henley , in an effort to smooth over a <mask> between the two one-time suitors for Santa Fe , a Chicago-based railroad and energy concern . 


{'bart': 'potential\ndispute\npossible\nrift\nrecent',
 'bert': 'settlement\nrelationship\ndispute\ndeal\nmeeting',
 'electra': 'dispute\nrelationship\nrift\ngap\ndeal',
 'roberta': 'dispute\ndisagreement\nfeud\nspat\nrift',
 'xlm': 'dispute\ndeal\nconflict\nrelationship\ntrade',
 'xlnet': 'dispute\nrow\nconflict\ndisagreement\ndifferences'}

In [ ]:
text_sentence = "Karen Stern has played the <mask> surrounded by ice sculptures at a restaurant and as part of the display for a three-foot-high cake at a pastry competition  "
get_all_predictions(text_sentence)

Karen Stern has played the <mask> surrounded by ice sculptures at a restaurant and as part of the display for a three-foot-high cake at a pastry competition  


{'bart': 'role\nviolin\npart\npiano\nice',
 'bert': 'song\npiano\nrole\nviolin\npart',
 'electra': 'role\nstage\ngame\npiano\npart',
 'roberta': 'violin\npiano\nguitar\npart\nrole',
 'xlm': 'guitar\npiano\nmusic\ngame\nstage',
 'xlnet': 'game\nstage\nfield\ntable\nturn'}

In [ ]:
text_sentence = "The Pac Man defense is named after the video game in which characters try to eat other characters before they get eaten <mask> . "
get_all_predictions(text_sentence)

The Pac Man defense is named after the video game in which characters try to eat other characters before they get eaten <mask> . 


{'bart': 'by\nthemselves\nand\ntoo\nor',
 'bert': 'alive\nagain\nfirst\nthemselves\nwhole',
 'electra': 'up\nagain\nout\nanyway\noff',
 'roberta': 'by\ntoo\nagain\nthemselves\n...',
 'xlm': 'by\nagain\n...\nout\nup',
 'xlnet': 'by\nalive\nand\nin\nback'}

In [ ]:
text_sentence = "Unself-consciously , the littlest cast member with the big voice steps into the audience in one number to open her wide cat-eyes and throat to melt the <mask> of one lucky patron each night . "
get_all_predictions(text_sentence)

Unself-consciously , the littlest cast member with the big voice steps into the audience in one number to open her wide cat-eyes and throat to melt the <mask> of one lucky patron each night . 


{'bart': 'hearts\nheart\nfrozen\nice\neyes',
 'bert': 'heart\ntongue\nsmile\nhearts\neyes',
 'electra': 'heart\neyes\nmind\nblood\nbody',
 'roberta': 'heart\nhearts\nsmile\nsoul\nice',
 'xlm': 'heart\nblood\neyes\nface\nhands',
 'xlnet': 'heart\nhearts\neyes\nthroat\nears'}

In [ ]:
text_sentence = "They have had to learn through the press about the sweet terms he had struck <mask> himself and his small management group "
get_all_predictions(text_sentence)

They have had to learn through the press about the sweet terms he had struck <mask> himself and his small management group 


{'bart': 'with\nfor\nin\nto\non',
 'bert': 'between\nfor\nwith\nto\non',
 'electra': 'for\non\nbetween\nwith\nupon',
 'roberta': 'between\nwith\nfor\non\namong',
 'xlm': 'for\nbetween\nabout\nwith\non',
 'xlnet': 'himself\nupon\nstruck\nin\nbetween'}

In [ ]:
text_sentence = "It was a fight that filled magazines and <mask> and gave professors case histories for business schools "
get_all_predictions(text_sentence)

It was a fight that filled magazines and <mask> and gave professors case histories for business schools 


{'bart': 'newspapers\nbooks\nnews\nblogs\nwebsites',
 'bert': 'newspapers\npapers\nbooks\nnewspaper\njournals',
 'electra': 'magazines\nbooks\nnewspapers\npapers\njournals',
 'roberta': 'newspapers\nbooks\ntextbooks\nwebsites\npapers',
 'xlm': 'books\nnewspaper\nwebsites\nradio\npaper',
 'xlnet': 'and\nbooks\ndid\ncolleges\nuniversities'}

In [ ]:
text_sentence = "To air fresh shows through June , `` we \'d have to eat the second <mask> of some episodes , \'\' he said , which would cause a financial hit . "
get_all_predictions(text_sentence)

To air fresh shows through June , `` we 'd have to eat the second <mask> of some episodes , '' he said , which would cause a financial hit . 


{'bart': 'half\nand\nor\nseason\nweek',
 'bert': 'half\npart\nseason\nquarter\nhalves',
 'electra': 'half\npart\nnight\nday\none',
 'roberta': 'half\npart\nquarter\nweek\nseason',
 'xlm': 'half\nseason\nweek\npart\nday',
 'xlnet': 'half\nout\nfirst\nthird\nepisode'}

In [ ]:
text_sentence = "The lower court had enjoined the effect of its decision , pending the appeal , but the appeals court dissolved the <mask> . "
get_all_predictions(text_sentence)

The lower court had enjoined the effect of its decision , pending the appeal , but the appeals court dissolved the <mask> . 


{'bart': 'injunction\norder\nen\nstay\nlower',
 'bert': 'appeal\ncase\ncourt\nproceedings\nsentence',
 'electra': 'decision\nruling\nappeal\ncase\nproceedings',
 'roberta': 'injunction\norder\nstay\nappeal\nsuspension',
 'xlm': 'appeal\neffect\ndecision\naction\ncase',
 'xlnet': 'order\njudgment\nproceedings\ncase\nappeal'}

In [ ]:
text_sentence = "Mr. Hauke `` is looking for a much more costly repair that is nonstandard and would knock the <mask> beyond the the { $ 500} limit for a buy-back . \'\' "
get_all_predictions(text_sentence)

Mr. Hauke `` is looking for a much more costly repair that is nonstandard and would knock the <mask> beyond the the { $ 500} limit for a buy-back . '' 


{'bart': 'price\nvalue\nstock\ncompany\ncost',
 'bert': 'company\ncity\nfirm\nbusiness\nmarket',
 'electra': 'price\nmoney\ncost\nprices\njob',
 'roberta': 'stock\ncompany\nshares\nprice\nstake',
 'xlm': 'door\nprice\nhouse\ndealer\nmarket',
 'xlnet': 'price\ncost\ncompany\nvalue\nrepair'}

In [ ]:
text_sentence = "They are irate about new capital-adequacy requirements that force securities firms to pump at least 20 <mask> more capital into reserves "
get_all_predictions(text_sentence)

They are irate about new capital-adequacy requirements that force securities firms to pump at least 20 <mask> more capital into reserves 


{'bart': 'percent\nper\ntimes\nbillion\npercentage',
 'bert': 'times\npercent\nyears\nor\nmillion',
 'electra': 'percent\ntimes\nor\nyears\nto',
 'roberta': 'percent\ntimes\nx\nX\npc',
 'xlm': 'percent\ntimes\nx\nyears\nX',
 'xlnet': 'more\npercent\n20\nbillion\nfurther'}

In [ ]:
text_sentence = "When the chairman of Santa Fe Southern Pacific Corp. flew into town a few months ago in the corporate jet , a small group of trainmen here knew they had struck a raw <mask> ."
get_all_predictions(text_sentence)

When the chairman of Santa Fe Southern Pacific Corp. flew into town a few months ago in the corporate jet , a small group of trainmen here knew they had struck a raw <mask> .


{'bart': 'nerve\ndeal\nchord\nnote\npolitical',
 'bert': 'nerve\ndeal\nmatch\nblow\nbargain',
 'electra': 'nerve\ndeal\nearthquake\nbullet\ndisaster',
 'roberta': 'deal\nnerve\npoint\nchord\nbargain',
 'xlm': 'deal\nfire\nmoment\ndecision\nbomb',
 'xlnet': 'deal\nnerve\nR\none\npoint'}

In [ ]:
text_sentence = "Meanwhile , the discounters and specialty stores that have grabbed <mask> from Sears in recent years are n\'t likely to stand still while it tries to catch up ."
get_all_predictions(text_sentence)

Meanwhile , the discounters and specialty stores that have grabbed <mask> from Sears in recent years are n't likely to stand still while it tries to catch up .


{'bart': 'market\na\nmore\nthe\ncustomers',
 'bert': 'it\norders\nspace\naway\ninventory',
 'electra': 'attention\nmoney\nprofits\naway\nrevenue',
 'roberta': 'ground\nshare\nbusiness\ncash\nshares',
 'xlm': 'customers\nattention\nmoney\ncash\nrevenue',
 'xlnet': 'up\ngrabbed\naway\nfrom\nmoney'}

In [ ]:
text_sentence = "In recent years , Drexel has poured far more <mask> into areas such as high-yield , `` junk \'\' bonds , merger and acquisition advice and mortgage-backed securities ."
get_all_predictions(text_sentence)

In recent years , Drexel has poured far more <mask> into areas such as high-yield , `` junk '' bonds , merger and acquisition advice and mortgage-backed securities .


{'bart': 'of\nmoney\nthan\ncapital\nresources',
 'bert': 'capital\nresources\nmoney\nexpertise\nattention',
 'electra': 'money\ncash\nresources\ncapital\nenergy',
 'roberta': 'money\nresources\ncapital\ncash\nfunds',
 'xlm': 'attention\nmoney\nexpertise\ninvestment\nresearch',
 'xlnet': 'resources\nmoney\nattention\nresearch\nexpertise'}

In [ ]:
text_sentence = "FIAT S.P.A . , the Italian auto maker , said it agreed to step up its <mask> with CIA. TELEFONICA NACIONAL DE ESPANA S.A. of Spain in the manufacturing of telecommunications equipment ."
get_all_predictions(text_sentence)

FIAT S.P.A . , the Italian auto maker , said it agreed to step up its <mask> with CIA. TELEFONICA NACIONAL DE ESPANA S.A. of Spain in the manufacturing of telecommunications equipment .


{'bart': 'cooperation\nefforts\nwork\ninvestment\njoint',
 'bert': 'cooperation\ncollaboration\npartnership\nrelationship\ninvolvement',
 'electra': 'partnership\ncooperation\ncollaboration\nrelationship\nrole',
 'roberta': 'cooperation\ncollaboration\npartnership\nwork\nactivities',
 'xlm': 'partnership\nrelationship\ncollaboration\nbusiness\nrelationships',
 'xlnet': 'cooperation\ncollaboration\npartnership\nrelations\nwork'}

In [ ]:
text_sentence = "But Mr. Sherlund said he thinks the company has taken steps to fix that <mask> ."
get_all_predictions(text_sentence)

But Mr. Sherlund said he thinks the company has taken steps to fix that <mask> .


{'bart': 'problem\nissue\nand\nsituation\nin',
 'bert': 'problem\nissue\nsituation\nmess\nthing',
 'electra': 'problem\nissue\nmess\nsituation\nscandal',
 'roberta': 'problem\nissue\nflaw\ndefect\nmistake',
 'xlm': 'problem\nissue\nsituation\nbug\nerror',
 'xlnet': 'that\nproblem\na\ntoo\nissue'}